In [1]:
import pandas as pd
import itertools
import numpy as np

In [2]:
csv= '/home/mei/nas/docker/thesis/data/csv/'

In [16]:
print('==> Loading data from timeseries files...')
timeseries_lab = pd.read_csv(csv + 'timeserieslab.csv', low_memory=False)
timeseries_periodic = pd.read_csv(csv+ 'timeseriesperiodic.csv')

==> Loading data from timeseries files...


In [17]:
print("there are {} patients in the  and {} records in lab table.".format(len(list(timeseries_lab ['patientunitstayid'].unique())),len(timeseries_lab)))
print("there are {} patients in the  and {} records in vital periodic table.".format(len(list(timeseries_periodic ['patientunitstayid'].unique())),len(timeseries_periodic)))


there are 10496 patients in the  and 794119 records in lab table.
there are 10496 patients in the  and 2905327 records in vital periodic table.


In [18]:
def reconfigure_timeseries(timeseries, offset_column, feature_column=None, test=False):
    """
    Reconfigure timeseries data by setting multi-index and pivoting if necessary.
    """
    if test:
        timeseries = timeseries.iloc[:5000]  # Limit for testing
    timeseries.set_index(['patientunitstayid', pd.to_timedelta(timeseries[offset_column], unit='min')], inplace=True)
    timeseries.drop(columns=offset_column, inplace=True)
    if feature_column:
        timeseries = timeseries.pivot_table(columns=feature_column, index=timeseries.index)
    timeseries.index = pd.MultiIndex.from_tuples(timeseries.index, names=['patient', 'time'])
    return timeseries

In [19]:
test=False
print('==> Reconfiguring lab timeseries...')
timeseries_lab = reconfigure_timeseries(timeseries_lab, 'labresultoffset', 'labname', test)
timeseries_lab.columns = timeseries_lab.columns.droplevel()

print('==> Reconfiguring periodic timeseries...')
timeseries_periodic = reconfigure_timeseries(timeseries_periodic, 'observationoffset', test=test)

print('==> Combining data together...')
merged = pd.concat([timeseries_lab, timeseries_periodic], axis=0, sort=False)

==> Reconfiguring lab timeseries...
==> Reconfiguring periodic timeseries...
==> Combining data together...


In [26]:
merged

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                   
252784  -1 days +20:55:00     NaN   NaN      NaN     NaN     NaN         NaN   
        -1 days +21:20:00     0.8   0.4     16.6     4.4     NaN        34.0   
        0 days 00:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 00:39:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
...                           ...   ...      ...     ...     ...         ...   
3347496 0 days 23:12:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:07:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:02:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:32:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:52:00       NaN   NaN      NaN     NaN     NaN         NaN   

                           AST (SGOT)   BUN  Base Excess  FiO2  ...  sao2  \
patient time                                                    ...         
252784  -1 days +20:55:00         NaN   NaN          NaN   NaN  ...   NaN   
        -1 days +21:20:00        24.0  22.0          NaN   NaN  ...   NaN   
        0 days 00:20:00           NaN   NaN        -25.6  28.0  ...   NaN   
        0 days 00:39:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 01:20:00           NaN   NaN          NaN   NaN  ...   NaN   
...                               ...   ...          ...   ...  ...   ...   
3347496 0 days 23:12:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:07:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:02:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:32:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 08:52:00           NaN   NaN          NaN   NaN  ...  98.0   

                           heartrate  respiration  cvp  systemicsystolic  \
patient time                                                               
252784  -1 days +20:55:00        NaN          NaN  NaN               NaN   
        -1 days +21:20:00        NaN          NaN  NaN               NaN   
        0 days 00:20:00          NaN          NaN  NaN               NaN   
        0 days 00:39:00          NaN          NaN  NaN               NaN   
        0 days 01:20:00          NaN          NaN  NaN               NaN   
...                              ...          ...  ...               ...   
3347496 0 days 23:12:00         86.0         16.0  NaN               NaN   
        0 days 23:07:00         88.0         24.0  NaN               NaN   
        0 days 23:02:00         86.0         24.0  NaN               NaN   
        0 days 23:32:00         81.0         19.0  NaN               NaN   
        0 days 08:52:00         91.0          NaN  NaN               NaN   

                           systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                               
252784  -1 days +20:55:00                NaN           NaN  NaN  NaN  NaN  
        -1 days +21:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:20:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 00:39:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                  NaN           NaN  NaN  NaN  NaN  
...                                      ...           ...  ...  ...  ...  
3347496 0 days 23:12:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:07:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:02:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:32:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 08:52:00                  NaN           NaN  NaN  NaN  NaN  

[29534

In [9]:
possible_value_ranges = {
   "temperature": (32,43),        # Temperature
    "sao2": (40, 100),             # SpO2
    "heartrate": (30, 400),        # Heart rate
    "respiration": (0, 60),        # Resp. rate
    "cvp": (0, 20),                # CVP
    
    # "etco2": (0, 60),             # EtCO2
    "systemicsystolic": (40, 300), # BP systolic
    "systemicdiastolic": (20, 150),# BP diastolic
    "systemicmean": (30, 200),     # BP mean
    
}

In [30]:
def filter_vital_signs(data, ranges):
    for column, (min_val, max_val) in ranges.items():
        if column in data.columns:
            data = data[(data[column].isna()) | ((data[column] >= min_val) & (data[column] <= max_val))]
    return data

merged = filter_vital_signs(merged, possible_value_ranges)
print("select valid vital signs")
print("There are {} patients and {} records in the vital periodic table.".format(
    merged.index.get_level_values('patient').nunique(),
    len(merged)
))

select valid vital signs
There are 10496 patients and 2953499 records in the vital periodic table.


In [31]:
merged

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                   
252784  -1 days +20:55:00     NaN   NaN      NaN     NaN     NaN         NaN   
        -1 days +21:20:00     0.8   0.4     16.6     4.4     NaN        34.0   
        0 days 00:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 00:39:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
...                           ...   ...      ...     ...     ...         ...   
3347496 0 days 23:12:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:07:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:02:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:32:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:52:00       NaN   NaN      NaN     NaN     NaN         NaN   

                           AST (SGOT)   BUN  Base Excess  FiO2  ...  sao2  \
patient time                                                    ...         
252784  -1 days +20:55:00         NaN   NaN          NaN   NaN  ...   NaN   
        -1 days +21:20:00        24.0  22.0          NaN   NaN  ...   NaN   
        0 days 00:20:00           NaN   NaN        -25.6  28.0  ...   NaN   
        0 days 00:39:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 01:20:00           NaN   NaN          NaN   NaN  ...   NaN   
...                               ...   ...          ...   ...  ...   ...   
3347496 0 days 23:12:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:07:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:02:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:32:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 08:52:00           NaN   NaN          NaN   NaN  ...  98.0   

                           heartrate  respiration  cvp  systemicsystolic  \
patient time                                                               
252784  -1 days +20:55:00        NaN          NaN  NaN               NaN   
        -1 days +21:20:00        NaN          NaN  NaN               NaN   
        0 days 00:20:00          NaN          NaN  NaN               NaN   
        0 days 00:39:00          NaN          NaN  NaN               NaN   
        0 days 01:20:00          NaN          NaN  NaN               NaN   
...                              ...          ...  ...               ...   
3347496 0 days 23:12:00         86.0         16.0  NaN               NaN   
        0 days 23:07:00         88.0         24.0  NaN               NaN   
        0 days 23:02:00         86.0         24.0  NaN               NaN   
        0 days 23:32:00         81.0         19.0  NaN               NaN   
        0 days 08:52:00         91.0          NaN  NaN               NaN   

                           systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                               
252784  -1 days +20:55:00                NaN           NaN  NaN  NaN  NaN  
        -1 days +21:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:20:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 00:39:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                  NaN           NaN  NaN  NaN  NaN  
...                                      ...           ...  ...  ...  ...  
3347496 0 days 23:12:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:07:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:02:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:32:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 08:52:00                  NaN           NaN  NaN  NaN  NaN  

[29534

In [32]:
def gen_patient_chunk(patients, merged, size=500): # 500 is a good size for LSTM
    """
    Generate patient data chunks for processing.
    """
    it = iter(patients)
    chunk = list(itertools.islice(it, size))
    while chunk:
        yield merged.loc[chunk]
        chunk = list(itertools.islice(it, size))

In [33]:
test=False


def resample_and_mask(timeseries, eICU_path, header, mask_decay=True, decay_rate=4/3, test=False, verbose=False):
    """
    Resample timeseries data to hourly intervals and calculate masking features.
    """
    if verbose:
        print('Resampling to 1 hour intervals...')
    timeseries = timeseries.groupby(level=[0, 1]).mean()
    unstacked = timeseries.unstack(level=0)
    unstacked.index = unstacked.index.ceil(freq='h')
    resampled = unstacked.resample('h', closed='right', label='right').mean()

    if mask_decay:
        if verbose:
            print('Calculating mask decay features...')
        mask_bool = resampled.notnull()
        mask = mask_bool.astype(int)
        mask.replace({0: np.nan}, inplace=True) 
        inv_mask_bool = ~mask_bool
        count_non_measurements = inv_mask_bool.cumsum() - inv_mask_bool.cumsum().where(mask_bool).ffill().fillna(0)
        mask = mask.ffill().fillna(0) / (count_non_measurements * decay_rate).replace(0, 1)
        mask = mask.iloc[-24:]
    else:
        if verbose:
            print('Calculating binary mask features...')
        mask = resampled.iloc[-24:].notnull().astype(int)

    if verbose:
        print('Filling missing data forwards...')
    resampled = resampled.ffill().iloc[-24:]
    resampled.index = list(range(1, 25))
    mask.index = list(range(1, 25))

    if verbose:
        print('Filling in remaining values with zeros...')
    resampled.fillna(0, inplace=True)

    if verbose:
        print('Reconfiguring and combining features with mask features...')
        
    resampled = resampled.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    mask = mask.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    
    mask.columns = [str(col) + '_mask' for col in mask.columns]
    final = pd.concat([resampled, mask], axis=1)

    if verbose:     # if verbose=True, print the progress
        print('Saving progress...')
    if test==False:  # if test=False, save the progress
        final.to_csv(csv + 'preprocessed_timeseries.csv', mode='a', header=header)

In [34]:
import gc

patients = merged.index.unique(level=0)
gen_chunks = gen_patient_chunk(patients, merged)
header = True
print('==> Initiating main processing loop...')
for i, patient_chunk in enumerate(gen_chunks, start=1):
    final=resample_and_mask(patient_chunk, csv, header, mask_decay=True, decay_rate=4/3, test=test, verbose=False)
    print(f'==> Processed {i * 500} patients...')
    header = False
    
    # Clear memory
    del patient_chunk
    gc.collect()

==> Initiating main processing loop...
==> Processed 500 patients...
==> Processed 1000 patients...
==> Processed 1500 patients...
==> Processed 2000 patients...
==> Processed 2500 patients...
==> Processed 3000 patients...
==> Processed 3500 patients...
==> Processed 4000 patients...
==> Processed 4500 patients...
==> Processed 5000 patients...
==> Processed 5500 patients...
==> Processed 6000 patients...
==> Processed 6500 patients...
==> Processed 7000 patients...
==> Processed 7500 patients...
==> Processed 8000 patients...
==> Processed 8500 patients...
==> Processed 9000 patients...
==> Processed 9500 patients...
==> Processed 10000 patients...
==> Processed 10500 patients...


In [6]:
processed_timeseries=pd.read_csv(csv + 'final_timeseries.csv')

In [7]:
processed_timeseries.rename(columns={'Unnamed: 1': 'time'}, inplace=True)
processed_timeseries.set_index('patient', inplace=True)

In [12]:
print("There are {} patients and {} records in the time series table.".format(
    processed_timeseries.index.nunique(),
    len(processed_timeseries)
))

There are 10023 patients and 240552 records in the time series table.


In [11]:
processed_timeseries

,time,-basos,-eos,-lymphs,-monos,-polys,ALT (SGPT),AST (SGOT),BUN,Base Excess,...,sao2_mask,heartrate_mask,respiration_mask,cvp_mask,systemicsystolic_mask,systemicdiastolic_mask,systemicmean_mask,st1_mask,st2_mask,st3_mask
patient,,,,,,,,,,,,,,,,,,,,,
252784,1,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,-25.6,...,1.000,1.00,1.000,0.0,0.0,0.0,0.0,1.00,1.00,1.00
252784,24,0.2,0.3,12.3,12.1,0.0,22.0,15.0,13.0,-25.6,...,0.750,0.75,0.750,0.0,0.0,0.0,0.0,0.75,0.75,0.75
252784,23,0.2,0.3,12.3,12.1,0.0,22.0,15.0,13.0,-25.6,...,1.000,1.00,1.000,0.0,0.0,0.0,0.0,1.00,1.00,1.00
252784,22,0.2,0.3,12.3,12.1,0.0,22.0,15.0,13.0,-25.6,...,1.000,1.00,1.000,0.0,0.0,0.0,0.0,1.00,1.00,1.00
252784,21,0.2,0.3,12.3,12.1,0.0,22.0,15.0,13.0,-25.6,...,1.000,1.00,1.000,0.0,0.0,0.0,0.0,1.00,1.00,1.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347496,2,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,2.0,...,1.000,1.00,0.750,0.0,0.0,0.0,0.0,0.00,0.00,0.00
3347496,1,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,2.0,...,1.000,1.00,1.000,0.0,0.0,0.0,0.0,0.00,0.00,0.00
3347496,23,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,0.0,...,0.750,1.00,1.000,0.0,0.0,0.0,0.0,0.00,0.00,0.00


In [9]:
flat_features = pd.read_csv(csv + 'final_flat_drug.csv')
flat_features.set_index('patient', inplace=True)

In [10]:
flat_features

,hour,gender,age,admissionweight,dischargeweight,> 89,ACETAMINOPHEN,ADVAIR DISKUS,ALBUTEROL,ALBUTEROL SULFATE,...,TRAMADOL HCL,TRAZODONE HCL,TYLENOL,VITAMIN C,VITAMIN D,VITAMIN D3,WARFARIN SODIUM,XANAX,ZOCOR,ZOFRAN
patient,,,,,,,,,,,,,,,,,,,,,
252784,15.0,1,56.0,0.261051,0.262847,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
253331,1.0,1,76.0,0.399234,0.397222,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
255112,5.0,1,52.0,0.176123,0.202083,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
258354,19.0,0,61.0,0.258893,0.261458,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
259414,1.0,1,81.0,0.374173,0.370486,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247094,21.0,1,22.0,0.317438,0.313542,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3247116,21.0,0,52.0,0.214758,0.236111,0,1,1,1,0,...,0,0,0,0,0,0,0,0,0,0
3247421,1.0,1,59.0,0.265228,0.285069,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
def add_time_of_day(processed_timeseries, flat_features):
    """
    Add time-of-day features to timeseries data.
    """
    print('==> Adding time of day features...')
    processed_timeseries = processed_timeseries.join(flat_features[['hour']], how='inner', on='patient')
    processed_timeseries['hour'] = processed_timeseries['time'] + processed_timeseries['hour'] # 计算绝对时间
    hour_list = np.linspace(0, 1, 24) # 生成 24 小时的归一化列表
    processed_timeseries['hour'] = processed_timeseries['hour'].apply(lambda x: hour_list[int(x % 24)]) 
    return processed_timeseries

In [14]:
processed_timeseries = add_time_of_day(processed_timeseries, flat_features)

==> Adding time of day features...


In [16]:
print('==> Removing non-varying time series features...then add them to flat features...')
mask_cols = [col for col in processed_timeseries.columns if 'mask' in col] # select columns with mask
mean_masks = processed_timeseries[mask_cols].eq(1).groupby('patient').mean().mean() # 计算 mask 列的激活比例
mask_to_flat = list(mean_masks[mean_masks <= 2 / 24].index) # 筛选出掩码激活比例小于等于 2/24 的列。这些掩码列表示的时间序列特征在大多数患者中几乎没有变化
cols_to_flat = [col[:-5] for col in mask_to_flat] + mask_to_flat #包含掩码激活比例较低的特征列和其对应的掩码列。['feature2', 'feature2_mask']

==> Removing non-varying time series features...then add them to flat features...


In [17]:
flat_features = flat_features.join(
        processed_timeseries.loc[processed_timeseries['time'] == 24][cols_to_flat], how='inner', on='patient' #使用 inner join，确保只保留两边都存在 patient 的数据
    )
processed_timeseries.drop(cols_to_flat, axis=1, inplace=True) # 从时间序列数据中移除不需要的列

processed_timeseries.drop(columns=['RR (patient)', 'RR (patient)_mask'], inplace=True, errors='ignore') # 删除额外的列， RR 是呼吸频率

In [19]:
flat_features

,hour,gender,age,admissionweight,dischargeweight,> 89,ACETAMINOPHEN,ADVAIR DISKUS,ALBUTEROL,ALBUTEROL SULFATE,...,sodium_mask,total bilirubin_mask,total protein_mask,troponin - I_mask,urinary specific gravity_mask,temperature_mask,cvp_mask,systemicsystolic_mask,systemicdiastolic_mask,systemicmean_mask
patient,,,,,,,,,,,,,,,,,,,,,
252784,15.0,1,56.0,0.261051,0.262847,0,0,0,0,0,...,0.187500,0.187500,0.187500,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
253331,1.0,1,76.0,0.399234,0.397222,0,0,0,1,0,...,0.044118,0.044118,0.044118,0.000,0.000000,0.0,0.000000,0.044118,0.044118,0.044118
255112,5.0,1,52.0,0.176123,0.202083,0,0,0,0,0,...,0.041667,0.041667,0.041667,0.000,0.046875,0.0,0.053571,0.750000,0.750000,0.750000
258354,19.0,0,61.0,0.258893,0.261458,0,0,0,0,0,...,0.083333,0.083333,0.083333,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
259414,1.0,1,81.0,0.374173,0.370486,0,0,0,0,0,...,0.050000,0.000000,0.000000,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3247094,21.0,1,22.0,0.317438,0.313542,0,0,0,0,0,...,0.068182,0.015957,0.015957,0.000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
3247116,21.0,0,52.0,0.214758,0.236111,0,1,1,1,0,...,0.057692,0.000000,0.000000,0.000,0.046875,0.0,0.000000,0.750000,0.750000,0.750000
3247421,1.0,1,59.0,0.265228,0.285069,0,0,0,0,0,...,0.050000,0.035714,0.035714,0.000,0.000000,0.0,0.750000,0.000000,0.000000,0.000000


In [18]:
processed_timeseries

,time,bedside glucose,sao2,heartrate,respiration,st1,st2,st3,bedside glucose_mask,sao2_mask,heartrate_mask,respiration_mask,st1_mask,st2_mask,st3_mask,hour
patient,,,,,,,,,,,,,,,,
252784,1,422.0,98.833333,120.416667,25.000000,0.058333,-0.166667,-0.225000,1.000,1.000,1.00,1.000,1.00,1.00,1.00,0.695652
252784,24,170.0,97.583333,97.416667,20.750000,-0.008333,-0.008333,-0.008333,0.750,0.750,0.75,0.750,0.75,0.75,0.75,0.652174
252784,23,170.0,97.583333,97.416667,20.750000,-0.008333,-0.008333,-0.008333,1.000,1.000,1.00,1.000,1.00,1.00,1.00,0.608696
252784,22,170.0,97.000000,88.083333,17.000000,-0.083333,0.025000,0.058333,1.000,1.000,1.00,1.000,1.00,1.00,1.00,0.565217
252784,21,226.0,97.166667,86.416667,16.500000,-0.075000,-0.008333,0.033333,1.000,1.000,1.00,1.000,1.00,1.00,1.00,0.521739
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3347496,2,0.0,98.166667,83.333333,30.000000,0.000000,0.000000,0.000000,0.000,1.000,1.00,0.750,0.00,0.00,0.00,0.260870
3347496,1,0.0,98.888889,83.444444,30.000000,0.000000,0.000000,0.000000,0.000,1.000,1.00,1.000,0.00,0.00,0.00,0.217391
3347496,23,105.0,95.100000,82.666667,21.166667,0.000000,0.000000,0.000000,0.750,0.750,1.00,1.000,0.00,0.00,0.00,0.130435


In [20]:
print('==> Saving flat features and final timeseries data after delete the low mask ratio...')
flat_features.to_csv(csv+ 'final_flat_del_low_mask_ratio.csv')
processed_timeseries.to_csv(csv + 'final_timeseries_del_low_mask_ratio.csv')

==> Saving flat features and final timeseries data after delete the low mask ratio...
